In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\lgarzia\Documents\GitHub\explainability\FBPS-ValidationData.csv', sep='\t')

In [3]:
df.drop(['dateload', 'submittime'], axis=1, inplace=True)

In [4]:
df.head(n=1)

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,OPN7,OPN8,OPN9,OPN10,country,source,screensize,introelapse,testelapse,endelapse
0,4,4,4,5,5,4,3,4,4,5,...,4,3,4,3,DE,1,2,3,6592,10


In [5]:
df = df[(df.country.isin(['US', 'GB', 'CA', 'AU', 'NZ', 'IE']))]
df = df[df.age <= 90]
df = df[(df.birthn > 1) & (df.birthn < 11) & (df.birthpos <= df.birthn) & (df.birthpos > 0)]
df = pd.get_dummies(df, columns=['gender', 'country', 'source']).rename({'gender_0':'gender_undefined', 'gender_1':'gender_male', 'gender_2':'gender_female', 'gender_3':'gender_other', 'source_0':'source_undefined', 'source_1':'source_google', 'source_2':'source_direct', 'source_3':'source_other'}, axis=1)
df.loc[:, 'birthorder'] = 0
df.loc[(df.birthpos == 1), 'birthorder'] = 1
df.loc[(df.birthpos > 1) & (df.birthpos < df.birthn), 'birthorder'] = 2
df.loc[(df.birthpos > 1) & (df.birthpos == df.birthn), 'birthorder'] = 3
df.drop(['birthpos'], axis=1, inplace=True)

In [6]:
df.head(n=5)

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,country_AU,country_CA,country_GB,country_IE,country_NZ,country_US,source_undefined,source_google,source_direct,birthorder
3,2,1,3,2,4,3,3,3,3,5,...,0,0,0,0,0,1,1,0,0,1
4,2,1,4,5,2,5,4,4,2,5,...,0,0,0,0,0,1,1,0,0,1
5,5,1,1,3,2,2,4,3,3,5,...,0,0,0,0,0,1,0,1,0,1
6,2,2,3,5,3,2,4,4,5,5,...,0,0,0,0,0,1,0,0,1,1
7,4,5,5,5,3,5,4,4,5,4,...,0,0,0,0,0,1,0,1,0,1


In [11]:
print(df.describe().T)

                    count      mean       std  min  25%  50%  75%  max
Q1                25813.0  2.771549  1.389309  0.0  2.0  3.0  4.0  5.0
Q2                25813.0  2.404602  1.644840  0.0  1.0  2.0  4.0  5.0
Q3                25813.0  3.479642  1.277640  0.0  3.0  4.0  4.0  5.0
Q4                25813.0  3.019293  1.413205  0.0  2.0  3.0  4.0  5.0
Q5                25813.0  3.008910  1.235201  0.0  2.0  3.0  4.0  5.0
...                   ...       ...       ...  ...  ...  ...  ...  ...
country_US        25813.0  0.734320  0.441703  0.0  0.0  1.0  1.0  1.0
source_undefined  25813.0  0.403789  0.490666  0.0  0.0  0.0  1.0  1.0
source_google     25813.0  0.407895  0.491453  0.0  0.0  0.0  1.0  1.0
source_direct     25813.0  0.188316  0.390972  0.0  0.0  0.0  0.0  1.0
birthorder        25813.0  1.920389  0.870363  1.0  1.0  2.0  3.0  3.0

[97 rows x 8 columns]


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
rand = 9
y = df['birthorder']
X = df.drop(['birthorder'], axis=1).copy()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=rand)

In [16]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics, linear_model

In [17]:
from sklearn import tree, discriminant_analysis, ensemble, neural_network, inspection

In [18]:

# https://pdpbox.readthedocs.io/en/latest/
import matplotlib.pyplot as plt
from pdpbox import pdp

In [ ]:
decision_tree = tree.DecisionTreeClassifier(max_depth=6, random_state=rand, class_weight='balanced')
gradient_boosting = ensemble.GradientBoostingClassifier(n_estimators=200, max_depth=4, subsample=.5, learning_rate=.05)
random_forest = ensemble.RandomForestClassifier(max_depth=11, n_estimators=300, max_features='sqrt', random_state=rand)
